In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chopin-music-dataset/chpn-p13.mid
/kaggle/input/chopin-music-dataset/chpn_op66.mid
/kaggle/input/chopin-music-dataset/chpn_op35_1.mid
/kaggle/input/chopin-music-dataset/chpn-p10.mid
/kaggle/input/chopin-music-dataset/chpn_op23.mid
/kaggle/input/chopin-music-dataset/chpn-p4.mid
/kaggle/input/chopin-music-dataset/chpn-p2.mid
/kaggle/input/chopin-music-dataset/chpn_op25_e2.mid
/kaggle/input/chopin-music-dataset/chpn_op35_2.mid
/kaggle/input/chopin-music-dataset/chpn_op27_2.mid
/kaggle/input/chopin-music-dataset/chpn-p7.mid
/kaggle/input/chopin-music-dataset/chpn_op33_4.mid
/kaggle/input/chopin-music-dataset/chpn-p1.mid
/kaggle/input/chopin-music-dataset/chpn-p8.mid
/kaggle/input/chopin-music-dataset/chpn-p17.mid
/kaggle/input/chopin-music-dataset/chpn_op10_e05.mid
/kaggle/input/chopin-music-dataset/chpn-p11.mid
/kaggle/input/chopin-music-dataset/chpn_op27_1.mid
/kaggle/input/chopin-music-dataset/chpn_op33_2.mid
/kaggle/input/chopin-music-dataset/chpn-p18.mid
/kaggle/input/ch

In [ ]:
import tensorflow as tf

# Check TPU availability and initialize the TPU system
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Resolve the TPU
    tf.config.experimental_connect_to_cluster(tpu)             # Connect to TPU
    tf.tpu.experimental.initialize_tpu_system(tpu)             # Initialize TPU system
    strategy = tf.distribute.TPUStrategy(tpu)                  # Create a strategy for TPU distribution
    print('Running on TPU:', tpu)
    
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy if TPU is not available
    print('Running on CPU/GPU')

print("REPLICAS:", strategy.num_replicas_in_sync)


In [2]:
pip install music21


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 29.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [6]:
filepath = "/kaggle/input/chopin-music-dataset/"
#Getting midi files
all_midis= []
print(os.listdir(filepath))
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)

['chpn-p13.mid', 'chpn_op66.mid', 'chpn_op35_1.mid', 'chpn-p10.mid', 'chpn_op23.mid', 'chpn-p4.mid', 'chpn-p2.mid', 'chpn_op25_e2.mid', 'chpn_op35_2.mid', 'chpn_op27_2.mid', 'chpn-p7.mid', 'chpn_op33_4.mid', 'chpn-p1.mid', 'chpn-p8.mid', 'chpn-p17.mid', 'chpn_op10_e05.mid', 'chpn-p11.mid', 'chpn_op27_1.mid', 'chpn_op33_2.mid', 'chpn-p18.mid', 'chpn-p15.mid', 'chpn_op25_e11.mid', 'chpn-p6.mid', 'chpn_op10_e01.mid', 'chpn_op25_e1.mid', 'chpn-p12.mid', 'chpn-p20.mid', 'chpn-p9.mid', 'chpn-p14.mid', 'chpn-p24.mid', 'chpn-p5.mid', 'chpn_op25_e12.mid', 'chpn_op25_e4.mid', 'chpn_op53.mid', 'chpn_op25_e3.mid', 'chpn-p16.mid', 'chpn-p23.mid', 'chpn_op10_e12.mid', 'chpn_op35_4.mid', 'chpn-p19.mid', 'chpn-p3.mid', 'chpn_op7_1.mid', 'chpn-p22.mid', 'chpn_op35_3.mid', 'chp_op31.mid', 'chp_op18.mid', 'chpn-p21.mid', 'chpn_op7_2.mid']


In [7]:
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

Total notes in all the Chopin midis in the dataset: 57894


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Activation
from keras.callbacks import EarlyStopping
# form keras.optimizer import Adams
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# model.fit(X, y, epochs=100, batch_size=265, validation_split=0.2, callbacks=[early_stopping])

with strategy.scope():  # Assuming you've set up TPU strategy earlier
    model = Sequential()
    model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(len(unique_notes)))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

# Train the model on TPU
history = model.fit(X, y, epochs=200, batch_size=64, validation_split=0.2)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 397)            │       102,029 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 397)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,385,229 (12.91 MB)

 Trainable params: 3,385,229 (12.91 MB)

 Non-trainable params: 0 (0.00 B)

...

723/723 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 4.9031

I0000 00:00:1726090073.479663     748 tpu_compile_op_common.cc:245] Compilation of bdcb9a51be458a6d:0:0 with session name  took 6.282984519s and succeeded
I0000 00:00:1726090073.503300     748 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(bdcb9a51be458a6d:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_10296812533627564416", property.function_library_fingerprint = 16689878318127575847, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,100,1,;4,397,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726090073.503333     748 tpu_compilation_cache_interface.cc:541] After adding entry for key bd

723/723 ━━━━━━━━━━━━━━━━━━━━ 45s 50ms/step - loss: 4.9029 - val_loss: 4.8986
Epoch 2/200


I0000 00:00:1726090078.198352     742 tpu_compile_op_common.cc:245] Compilation of d39f9fbd1238eb65:0:0 with session name  took 452.282225ms and succeeded
I0000 00:00:1726090078.201889     742 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d39f9fbd1238eb65:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_12840951802411055148", property.function_library_fingerprint = 3253389916127041468, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "5,100,1,;5,397,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726090078.201914     742 tpu_compilation_cache_interface.cc:541] After adding entry for key d39

  7/723 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - loss: 4.9457

In [8]:
from collections import Counter
import numpy as np

# Get all unique notes and chords
unique_notes = sorted(set(Corpus))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Create input sequences and corresponding outputs"
sequence_length = 100  # Adjust this based on your preference
input_sequences = []
output_notes = []

# Prepare sequences of notes
for i in range(len(Corpus) - sequence_length):
    sequence_in = Corpus[i:i + sequence_length]
    sequence_out = Corpus[i + sequence_length]
    input_sequences.append([note_to_int[note] for note in sequence_in])
    output_notes.append(note_to_int[sequence_out])

# Reshape input for LSTM
X = np.reshape(input_sequences, (len(input_sequences), sequence_length, 1))
X = X / float(len(unique_notes))  # Normalize input

# One-hot encode the output notes
y = np.eye(len(unique_notes))[output_notes]


In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Activation

# Define the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(unique_notes)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 397)            │       102,029 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 397)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,385,229 (12.91 MB)

 Trainable params: 3,385,229 (12.91 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation

# Create a MirroredStrategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()

# Print number of GPUs being used
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Build and compile the model inside the strategy scope
with strategy.scope():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(len(unique_notes)))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
history = model.fit(X, y, epochs=200, batch_size=265)


Number of devices: 2
Epoch 1/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 31s 106ms/step - loss: 5.1257
Epoch 2/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 24s 109ms/step - loss: 4.8480
Epoch 3/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 112ms/step - loss: 4.8216
Epoch 4/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 115ms/step - loss: 4.7994
Epoch 5/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.7097
Epoch 6/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.6964
Epoch 7/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.6812
Epoch 8/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.6787
Epoch 9/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.6690
Epoch 10/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step - loss: 4.6581
Epoch 11/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 115ms/step - loss: 4.6387
Epoch 12/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 115ms/step - loss: 4.6260
Epoch 13/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 115ms/step - loss: 4.5950
Epoch 14/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 25s 115ms/

In [ ]:
from music21 import stream, note, chord

def notes_to_midi(generated_notes, file_name='generated_music.mid'):
    midi_stream = stream.Stream()

    for note_str in generated_notes:
        if '.' in note_str or note_str.isdigit():
            # It's a chord
            chord_notes = [note.Note(int(n)) for n in note_str.split('.')]
            new_chord = chord.Chord(chord_notes)
            midi_stream.append(new_chord)
        else:
            # It's a note
            new_note = note.Note(note_str)
            midi_stream.append(new_note)

    midi_stream.write('midi', fp=file_name)

# Save generated music to a MIDI file
notes_to_midi(generated_notes)


In [13]:
# Save the model in .h5 format
model.save('music_generation_model.h5')


In [14]:
from keras.models import load_model

# Load the model from the saved .h5 file
model = load_model('music_generation_model.h5')


In [15]:
import numpy as np
from music21 import stream, note, chord

# Function to generate music
def generate_music(model, seed_sequence, num_notes, unique_notes):
    generated_notes = []
    pattern = seed_sequence

    for note_index in range(num_notes):
        # Reshape input to fit LSTM model (batch_size, sequence_length, input_dim)
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(unique_notes))

        # Predict the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Convert the predicted output into a note or chord
        index = np.argmax(prediction)
        result = unique_notes[index]
        generated_notes.append(result)

        # Append the result to the pattern for the next prediction
        pattern = np.append(pattern[1:], index)

    return generated_notes

# Example usage
seed_sequence = X[0]  # Take the first sequence from your data as a seed
generated_notes = generate_music(model, seed_sequence, num_notes=500, unique_notes=unique_notes)

# Convert generated notes back to MIDI
def create_midi(generated_notes):
    output_notes = []
    for pattern in generated_notes:
        if ('.' in pattern) or pattern.isdigit():  # It's a chord
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                notes.append(note.Note(int(current_note)))
            new_chord = chord.Chord(notes)
            output_notes.append(new_chord)
        else:  # It's a note
            new_note = note.Note(pattern)
            output_notes.append(new_note)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')

# Save generated music to MIDI file
create_midi(generated_notes)


In [ ]:
# Save the trained model in .h5 format
model.save('/kaggle/working/music_generation_model.h5')


In [ ]:
from keras.models import load_model

# Load the model from the saved .h5 file
model = load_model('/kaggle/working/music_generation_model.h5')


In [16]:
from IPython.display import FileLink

# Create a link to download the model
FileLink('/kaggle/working/music_generation_model.h5')


/kaggle/working/music_generation_model.h5

In [17]:
import numpy as np
from music21 import stream, note, chord
from keras.models import load_model

# Load the saved model
model = load_model('/kaggle/working/music_generation_model.h5')

# Function to generate music
def generate_music(model, seed_sequence, num_notes, unique_notes):
    generated_notes = []
    pattern = seed_sequence

    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(unique_notes))

        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = unique_notes[index]
        generated_notes.append(result)
        pattern = np.append(pattern[1:], index)

    return generated_notes

# Use a seed sequence to generate new music
seed_sequence = X[0]  # Example seed sequence from the training data
generated_notes = generate_music(model, seed_sequence, num_notes=500, unique_notes=unique_notes)

# Convert generated notes to MIDI
def create_midi(generated_notes):
    output_notes = []
    for pattern in generated_notes:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                notes.append(note.Note(int(current_note)))
            new_chord = chord.Chord(notes)
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            output_notes.append(new_note)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='/kaggle/working/generated_music.mid')

# Save generated music as MIDI
create_midi(generated_notes)


In [18]:
!pip install midi2audio


In [21]:
!pip install pydub
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [26]:
!pip install pretty_midi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592292 sha256=3bd450605354f7fd3a31ff920b75ddb7f63b652f3f57b14fc0ca4103aeffd938
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 require

In [32]:
#to play audio or corpus
IPython.display.Audio("/kaggle/working/generated_music.wav")